# OpenFinGuard — Time-Aware Train/Validation Split

Week: 4  
Day: 3  
Objective:
- Create a point-in-time modeling table
- Split data by time (no leakage)


Load features and labels

In [3]:
import pandas as pd

# Load raw tables
cycles = pd.read_csv(
    "../data/raw/billing_cycles.csv",
    parse_dates=["cycle_start_date", "cycle_end_date", "due_date"]
)

cards = pd.read_csv("../data/raw/credit_cards.csv")

transactions = pd.read_csv(
    "../data/raw/transactions.csv",
    parse_dates=["transaction_date"]
)


In [4]:
cycles = cycles.merge(
    cards[["card_id", "credit_limit"]],
    on="card_id",
    how="left"
)

cycles["credit_utilization"] = cycles["total_due"] / cycles["credit_limit"]
cycles["min_due_ratio"] = cycles["minimum_due"] / cycles["total_due"]


In [5]:
cycle_spend = (
    transactions.merge(
        cycles[["billing_cycle_id", "card_id", "cycle_start_date", "cycle_end_date"]],
        on="card_id",
        how="left"
    )
    .query("transaction_date >= cycle_start_date and transaction_date <= cycle_end_date")
    .groupby("billing_cycle_id")["amount"]
    .sum()
    .reset_index(name="monthly_spend")
)


In [6]:
features = cycles.merge(
    cycle_spend, on="billing_cycle_id", how="left"
)


In [7]:
monthly_card_spend = (
    transactions
    .groupby(["card_id", transactions["transaction_date"].dt.to_period("M")])["amount"]
    .sum()
    .reset_index()
)

card_volatility = (
    monthly_card_spend
    .groupby("card_id")["amount"]
    .std()
    .reset_index(name="spend_volatility")
)

features = features.merge(
    card_volatility, on="card_id", how="left"
)


In [8]:
features = features.sort_values(["card_id", "cycle_end_date"])

features["util_roll_3"] = (
    features.groupby("card_id")["credit_utilization"]
    .rolling(3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

features["spend_roll_3"] = (
    features.groupby("card_id")["monthly_spend"]
    .rolling(3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

features["spend_shock"] = (
    features["monthly_spend"] > 1.5 * features["spend_roll_3"]
)


In [9]:
import os
os.makedirs("../data/processed", exist_ok=True)


In [10]:
final_features = features[[
    "billing_cycle_id",
    "credit_utilization",
    "min_due_ratio",
    "monthly_spend",
    "spend_volatility",
    "util_roll_3",
    "spend_roll_3",
    "spend_shock"
]]

final_features.to_csv("../data/processed/features.csv", index=False)


In [11]:
features = pd.read_csv("../data/processed/features.csv")
labels = pd.read_csv("../data/processed/labels.csv")


In [14]:
#Build the modeling table
modeling = (
    features
    .merge(labels, on="billing_cycle_id", how="inner")
    .merge(
        cycles[["billing_cycle_id", "cycle_end_date"]],
        on="billing_cycle_id",
        how="left"
    )
)

modeling.head()




,billing_cycle_id,credit_utilization,min_due_ratio,monthly_spend,spend_volatility,util_roll_3,spend_roll_3,spend_shock,dpd,late_30,late_60,late_90,cycle_end_date
0,card_1_5,0.33414,0.119471,25800,15156.736676,0.334140,25800.000000,False,0,0,0,0,2025-09-04
1,card_1_4,0.54210,0.093636,27500,15156.736676,0.438120,26650.000000,False,0,0,0,0,2025-10-04
2,card_1_3,0.45668,0.094727,9600,15156.736676,0.444307,20966.666667,False,29,0,0,0,2025-11-03
3,card_1_2,0.34314,0.144606,40200,15156.736676,0.447307,25766.666667,True,0,0,0,0,2025-12-03
4,card_1_1,0.29862,0.124774,48300,15156.736676,0.366147,32700.000000,False,0,0,0,0,2026-01-02


In [15]:
modeling.isnull().mean()

billing_cycle_id      0.0
credit_utilization    0.0
min_due_ratio         0.0
monthly_spend         0.0
spend_volatility      0.0
util_roll_3           0.0
spend_roll_3          0.0
spend_shock           0.0
dpd                   0.0
late_30               0.0
late_60               0.0
late_90               0.0
cycle_end_date        0.0
dtype: float64

## Define the time-based split
We split by cycle_end_date.

4.1 Choose cutoff date

Use the latest 20–25% of time as validation.

In [16]:
cutoff_date = modeling["cycle_end_date"].quantile(0.75)
cutoff_date


Timestamp('2026-01-02 00:00:00')

4.2 Create splits

In [17]:
train = modeling[modeling["cycle_end_date"] <= cutoff_date]
valid = modeling[modeling["cycle_end_date"] > cutoff_date]

train.shape, valid.shape


((150, 13), (30, 13))

Explain in Markdown:

Training uses earlier cycles
Validation uses later cycles
Mimics real-world deployment

Verify no leakage

In [18]:
train["cycle_end_date"].max(), valid["cycle_end_date"].min()


(Timestamp('2026-01-02 00:00:00'), Timestamp('2026-02-01 00:00:00'))

Save modeling table

In [19]:
modeling.to_csv("../data/processed/modeling_table.csv", index=False)
